# Previsão se uma transação é fraudulenta

-> Desenvolvido por Rafael Santos

#### Qual problema de negócio quero resolver com essa solução?

- Identificar se as transações são fraudulentas ou não.

Para isso serão utilizadas as seguintes tabelas:

1.	accounts

Tabela que apresenta as informações cadastrais de cada conta.
Colunas:

-	id: ID identificador da tabela
-	account_number: Número da conta
-	birth: Data de nascimento
-	occupation: Tipo de negócio autodeclarado
-	email: E-mail da conta
-	address_id: ID identificador da tabela address
-	created_at: Data de criação da conta

2.	address

Tabela que identifica os pares de estado e cidade. Para verificar a residência de cada conta é necessário realizar o join com esta tabela.

-	id: ID identificador da tabela
-	state: Estado de residência do cliente
-	city: Cidade de residência do cliente

3.	levels

Cada conta recebe uma classificação de acordo com a forma que utiliza a plataforma. Contas que utilizam com maior consistência ou com grande potencial podem receber uma melhor classificação (A>B>C>D). Caso identifique-se que a conta possui características suspeitas, de fraude, é atribuída a categoria F e executado o encerramento.
-	id: ID identificador da tabela
-	account_number: Número da conta
-	level: A, B, C, D e F
-	created_at: Data da classificação

4.	charges

Tabela apresenta as emissões de boletos realizadas pelos clientes com os respectivos status de pago ou não.
-	id: ID identificador da tabela
-	account_number: Número da conta
-	status: Status da cobrança [paid, unpaid]
-	value: Valor da cobrança (em centavos)
-	created_at: Data de criação do boleto

5.	transactions

Tabela com as transações efetivadas por cada conta, logo, caso um boleto tenha sido pago esta informação estará presente nesta tabela e na tabela charges.
-	id: ID identificador da tabela
-	account_number: Número da conta
-	transaction_type_id: ID identificador da tabela transaction_type
-	value: Valor da transação (em centavos)
-	created_at: Data da transação

6.	transaction_type

Tabela que permite identificar qual o tipo de cada transação da tabela transactions.
-	id: ID identificador da tabela
-	description: boleto_recebido, pix_enviado e pix_recebido
-	description_long: 'BOLETO RECEBIDO PELO CLIENTE', 'PIX ENVIADO PELO CLIENTE PARA UMA CONTA EXTERNA' e 'PIX RECEBIDO PELO CLIENTE'


#### Imports necessários

In [1]:
import pandas as pd
import seaborn as sns

import sqlite3
con = sqlite3.connect('desafio-tecnico.db')

#### Visualizando os dados importados

In [2]:
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)
print(table)

               name
0          accounts
1           address
2            levels
3           charges
4      transactions
5  transaction_type


#### Criando os dataframes

In [3]:
df_accounts = pd.read_sql_query("SELECT * FROM accounts", con)
df_address = pd.read_sql_query("SELECT * FROM address", con)
df_levels = pd.read_sql_query("SELECT * FROM levels", con)
df_charges = pd.read_sql_query("SELECT * FROM charges", con)
df_transactions = pd.read_sql_query("SELECT * FROM transactions", con)
df_transaction_type = pd.read_sql_query("SELECT * FROM transaction_type", con)

#### Verificando se a importação foi correta

In [4]:
df_accounts.shape

(1029, 7)

In [5]:
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029 entries, 0 to 1028
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1029 non-null   int64 
 1   account_number  1029 non-null   int64 
 2   birth           1029 non-null   object
 3   occupation      1029 non-null   object
 4   email           1029 non-null   object
 5   address_id      1029 non-null   int64 
 6   created_at      1029 non-null   object
dtypes: int64(3), object(4)
memory usage: 56.4+ KB


In [6]:
df_accounts.head(1)

,id,account_number,birth,occupation,email,address_id,created_at
0,1,155938,1984-10-29,Outros,mariaceciliaazevedo042@example.com,1,2021-04-01 02:30:00


com base na lista de colunas descritas no cabeçalho, é possível verificar que a importação foi correta.

## Data Cleaning

#### O formato dos campos estão corretos?

In [7]:
print(df_accounts.info())
print("***************")
print("***************")
print(df_address.info())
print("***************")
print("***************")
print(df_levels.info())
print("***************")
print("***************")
print(df_charges.info())
print("***************")
print("***************")
print(df_transactions.info())
print("***************")
print("***************")
print(df_transaction_type.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029 entries, 0 to 1028
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1029 non-null   int64 
 1   account_number  1029 non-null   int64 
 2   birth           1029 non-null   object
 3   occupation      1029 non-null   object
 4   email           1029 non-null   object
 5   address_id      1029 non-null   int64 
 6   created_at      1029 non-null   object
dtypes: int64(3), object(4)
memory usage: 56.4+ KB
None
***************
***************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747 entries, 0 to 746
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          747 non-null    int64 
 1   state       747 non-null    object
 2   city        747 non-null    object
 3   created_at  747 non-null    object
dtypes: int64(1), object(3)
memory usage: 23.5+ KB
None
**********

#### Há valores nulos?

In [8]:
df_accounts.isnull().sum()

id                0
account_number    0
birth             0
occupation        0
email             0
address_id        0
created_at        0
dtype: int64

In [9]:
df_address.isnull().sum()

id            0
state         0
city          0
created_at    0
dtype: int64

In [10]:
df_levels.isnull().sum()

id                0
account_number    0
level             0
created_at        0
dtype: int64

In [11]:
df_charges.isnull().sum()

id                0
account_number    0
status            0
value             0
created_at        0
dtype: int64

In [12]:
df_transactions.isnull().sum()

id                     0
account_number         0
transaction_type_id    0
value                  0
created_at             0
dtype: int64

In [13]:
df_transaction_type.isnull().sum()

id                  0
description         0
description_long    0
dtype: int64

In [14]:
print(df_accounts.shape[0])
print(df_address.shape[0])
print(df_levels.shape[0])
print(df_charges.shape[0])
print(df_transactions.shape[0])
print(df_transaction_type.shape[0])

1029
747
926
154469
256227
3


Existem alguns campos que estão com o formato indevido, primeiramente, vou juntar as tabelas e depois fazer a transformação

In [15]:
df_transaction_type.head()

,id,description,description_long
0,1,boleto_recebido,BOLETO RECEBIDO PELO CLIENTE
1,2,pix_enviado,PIX ENVIADO PELO CLIENTE PARA UMA CONTA EXTERNA
2,3,pix_recebido,PIX RECEBIDO PELO CLIENTE


In [16]:
df = pd.merge(df_accounts,df_address[['id','state','city']],how='left',on='id')
df = pd.merge(df,df_levels[['id','level']],how='left',on='id')

alterei o nome da coluna para padronizar com a coluna que já tem criada

In [17]:
df_transaction_type = df_transaction_type.rename(columns={'id': 'transaction_type_id'})

In [18]:
df = pd.merge(df,df_charges[['id','status','value']],how='outer',on='id')
df = pd.merge(df,df_transactions[['id','transaction_type_id']],how='outer',on='id')
df = pd.merge(df,df_transaction_type[['transaction_type_id','description']],how='left',on='transaction_type_id')

In [19]:
df.head()

,id,account_number,birth,occupation,email,address_id,created_at,state,city,level,status,value,transaction_type_id,description
0,1,155938.0,1984-10-29,Outros,mariaceciliaazevedo042@example.com,1.0,2021-04-01 02:30:00,RJ,Carmo,F,paid,11992.0,1,boleto_recebido
1,2,747007.0,1977-03-26,Corretor,ananunes3@example.com,2.0,2021-04-01 02:00:00,MG,Bertopolis,D,paid,9689.0,1,boleto_recebido
2,3,208404.0,1984-01-26,Analista,luizfernandovieira@example.com,3.0,2021-04-01 11:30:00,MG,Olaria,NaN,paid,11579.0,1,boleto_recebido
3,4,66402.0,1982-06-30,Investidor,gabriellynascimento6@example.com,4.0,2021-04-01 14:30:00,PR,Icaraima,D,unpaid,3134.0,3,pix_recebido
4,5,277012.0,1967-01-11,Outros,aliciagoncalves7@example.com,5.0,2021-04-02 15:30:00,ES,Ecoporanga,D,paid,3085.0,3,pix_recebido


In [20]:
df.isnull().sum()

id                          0
account_number         255198
birth                  255198
occupation             255198
email                  255198
address_id             255198
created_at             255198
state                  255480
city                   255480
level                  255301
status                 101758
value                  101758
transaction_type_id         0
description                 0
dtype: int64

separei o que estava como nulo em um outra tabela

In [26]:
df_test = df[df['state'].isnull()]

In [31]:
df_treino = df.dropna()

verificando se ainda há nulos

In [32]:
df_treino.isnull().sum()

id                     0
account_number         0
birth                  0
occupation             0
email                  0
address_id             0
created_at             0
state                  0
city                   0
level                  0
status                 0
value                  0
transaction_type_id    0
description            0
dtype: int64